In [1]:
# Imports
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely.geometry import box
from itertools import cycle

import folium
from branca.element import Element

In [2]:
# Paths and map settings
DATA_DIR = Path("data")
TRIP_GEN_PATH = DATA_DIR / "trip_generators_brooklyn.geojson"
BUILDINGS_PATH = DATA_DIR / "buildings_brooklyn.geojson"

INSPECT_BOUNDS = {
    "min_lat": 40.671, "max_lat": 40.680,
    "min_lon": -73.980, "max_lon": -73.965,
}

ZOOM_START = 16

In [3]:
# Load data and ensure CRS
for path in [TRIP_GEN_PATH, BUILDINGS_PATH]:
    if not path.exists():
        raise FileNotFoundError(f"Missing input: {path}")

trip_generators_gdf = gpd.read_file(TRIP_GEN_PATH)
buildings_gdf = gpd.read_file(BUILDINGS_PATH)

building_required = {"building_id", "total_sqft", "estimated_floors"}
b_missing = building_required - set(buildings_gdf.columns)
if b_missing:
    raise ValueError(f"Buildings data missing required columns: {b_missing}")

required_cols = {"building_id", "sqft", "land_use_type", "source"}
missing = required_cols - set(trip_generators_gdf.columns)
if missing:
    raise ValueError(f"Trip generator data missing required columns: {missing}")

if buildings_gdf.crs is None:
    buildings_gdf = buildings_gdf.set_crs("EPSG:4326")
if trip_generators_gdf.crs is None:
    trip_generators_gdf = trip_generators_gdf.set_crs("EPSG:4326")

if buildings_gdf.crs != "EPSG:4326":
    buildings_gdf = buildings_gdf.to_crs("EPSG:4326")
if trip_generators_gdf.crs != "EPSG:4326":
    trip_generators_gdf = trip_generators_gdf.to_crs("EPSG:4326")

print(f"Loaded {len(buildings_gdf):,} buildings and {len(trip_generators_gdf):,} generators")

Loaded 331,538 buildings and 345,265 generators


In [ ]:
# Filter to inspection area and prepare attributes
bbox = box(INSPECT_BOUNDS["min_lon"], INSPECT_BOUNDS["min_lat"],
           INSPECT_BOUNDS["max_lon"], INSPECT_BOUNDS["max_lat"])
bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")

inspect_buildings = gpd.sjoin(buildings_gdf, bbox_gdf, how="inner", predicate="intersects").drop(columns=["index_right"])
inspect_generators = gpd.sjoin(trip_generators_gdf, bbox_gdf, how="inner", predicate="intersects").drop(columns=["index_right"])

print(f"Buildings in inspection area: {len(inspect_buildings):,}")
print(f"Generators / POIs in inspection area: {len(inspect_generators):,}")
print("Top land uses in inspection area:")
print(inspect_generators['land_use_type'].value_counts().head(15))

primary_use = inspect_generators.loc[inspect_generators.groupby('building_id')['sqft'].idxmax()].copy()
building_primary_use = primary_use[['building_id', 'land_use_type']].set_index('building_id')
inspect_buildings = inspect_buildings.join(building_primary_use, on='building_id')
inspect_buildings['primary_use'] = inspect_buildings['land_use_type'].fillna('unknown')

base_colors = {
    'residential': '#3388ff', 'restaurant': '#ff8c00', 'cafe': '#ffa500',
    'supermarket': '#228b22', 'convenience': '#32cd32', 'office': '#808080',
    'school': '#dc143c', 'place_of_worship': '#8b0000', 'industrial': '#2f4f4f',
    'unknown': '#cccccc'
}
fallback_palette = cycle(['#4c78a8', '#f58518', '#54a24b', '#b79a20', '#e45756', '#72b7b2', '#ff9da7', '#9d755d', '#bab0ac'])
for lu in inspect_buildings['primary_use'].dropna().unique():
    if lu not in base_colors:
        base_colors[lu] = next(fallback_palette)

inspect_buildings['color'] = inspect_buildings['primary_use'].map(base_colors).fillna('#999999')

popup_data = inspect_generators.groupby('building_id').apply(
    lambda grp: "<br>".join([
        f"<b>{row.land_use_type}</b>: {row.sqft:,.0f} sqft ({row.source})" + (f" – {row.name}" if pd.notna(row.name) else "")
        for _, row in grp.iterrows()
    ])
).to_dict()

inspect_buildings['popup_html'] = inspect_buildings.apply(
    lambda row: f'''
<b>Building ID:</b> {row.building_id}<br>
<b>Total SqFt:</b> {row.total_sqft:,.0f}<br>
<b>Floors:</b> {row.estimated_floors}<br>
<hr>
<b>Land Use Units:</b><br>
{popup_data.get(row.building_id, 'No POIs found')}
''',
    axis=1
)


In [ ]:
# Build map (buildings only) with legend
map_center = [bbox.centroid.y, bbox.centroid.x]
m = folium.Map(location=map_center, zoom_start=ZOOM_START, tiles="CartoDB positron")

folium.GeoJson(
    inspect_buildings[['geometry', 'color', 'popup_html']],
    style_function=lambda feature: {
        'fillColor': feature['properties']['color'],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    },
    popup=folium.GeoJsonPopup(fields=['popup_html'], labels=False, parse_html=True),
    name='Buildings'
).add_to(m)

legend_items = "".join([
    f"<div><span style='display:inline-block;width:12px;height:12px;background:{color};margin-right:6px;'></span>{label}</div>"
    for label, color in sorted(base_colors.items())
])
legend_html = f"""
<div style='position: fixed; bottom: 30px; left: 10px; z-index:9999; background: white; padding: 10px; border: 1px solid #ccc; border-radius: 4px; font-size: 12px;'>
<b>Primary use</b><br>{legend_items}
</div>
"""
m.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(m)
m